In [ ]:
import time
import csv

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [ ]:
driver = webdriver.Firefox()

In [ ]:

def open_front_page():
    driver.get("https://repertoire.sacem.fr/en/results?filters=parties&query=Eugene%20BOZZA#searchBtn", )

def accept_cookies():
    xpath_code = "//button[contains(text(), 'Accept Cookies')]"
    link = driver.find_element_by_xpath(xpath_code)
    link.click()
    
def click_show_following():
    xpath_code = "//button[contains(text(), 'show the following works')]"
    try:
        link = driver.find_element_by_xpath(xpath_code)
    except NoSuchElementException:
        return
    link.click()

def find_all_see_details():
    xpath_code = "//a[contains(text(), 'See detail')]"
    links = driver.find_elements_by_xpath(xpath_code)
    return links   
    
# def go_to_item_details():
#     driver.

In [ ]:
open_front_page()

In [ ]:
accept_cookies()

In [ ]:
for _ in range(20):
    click_show_following()
    time.sleep(1)

In [ ]:
all_detail_links = find_all_see_details()
print(len(all_detail_links))

In [ ]:
link_a = all_detail_links[0].get_attribute('href')
driver.execute_script(f"window.open('{link_a}', 'new_window')")

In [ ]:
whiteblc = driver.find_elements_by_xpath('//*[@data-reactid=".5.0"]')
print(len(whiteblc))
for clump in whiteblc:
    print(dir(clump))
    print(clump.get_attribute('class'))
    try:
        interested_rows = clump.find_elements_by_class_name('tooltip')
    except NoSuchElementException:
        pass
print(len(interested_rows))
# print(interested_rows[0].text)
# for row in interested_rows:
# interested_clumps = interested_rows.find_element_by_class_name()

# interested_parties = {'composer': }